In [1]:
from core.helpers.session_helper import SessionHelper
session = SessionHelper().session

import pandas as pd
import numpy as np
from datetime import datetime

2019-08-13 21:47:02,373 - core.helpers.session_helper.SessionHelper - INFO - Creating session for dev environment...
2019-08-13 21:47:02,397 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating administrator mocks.
2019-08-13 21:47:02,437 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating administrator mocks.
2019-08-13 21:47:02,438 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating pharmaceutical company mocks.
2019-08-13 21:47:02,443 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating pharmaceutical company mocks.
2019-08-13 21:47:02,445 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating brand mocks.
2019-08-13 21:47:02,449 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating brand mocks.
2019-08-13 21:47:02,451 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating segment mocks.
2019-08-13 21:47:0

In [2]:
"""
************ CONFIGURATION - PLEASE TOUCH **************
Pipeline Builder configuration: creates configurations from variables specified here!!
This cell will be off in production as configurations will come from the configuration postgres DB.
"""
# config vars: this dataset
config_pharma = "sun" # the pharmaceutical company which owns {brand}
config_brand = "ilumya" # the brand this pipeline operates on
config_state = "enrich" # the state this transform runs in
config_name = "accredo_cancel_before_active" # the name of this transform, which is the name of this notebook without .ipynb

# input vars: dataset to fetch. Recall that a contract published to S3 has a key format branch/pharma/brand/state/name
input_pharma = "sun"
input_brand = "ilumya"
input_state = "ingest"
input_name = "symphony_health_association_ingest_column_mapping"
input_branch = "sun-extract-validation" # if None, input_branch is automagically set to your working branch

In [3]:
"""
************ SETUP - DON'T TOUCH **************
Populating config mocker based on config parameters...
"""
import core.helpers.pipeline_builder as builder

ids = builder.build(config_pharma, config_brand, config_state, config_name, session)
transform_id = ids[0]
run_id = ids[1]

2019-08-13 21:47:03,362 - core.logging - DEBUG - Adding/getting mocks for specified configurations...
2019-08-13 21:47:03,387 - core.logging - DEBUG - Done. Creating mock run event and committing results to configuration mocker.


In [4]:
"""
************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.dataset_contract import DatasetContract

db_transform = session.query(Transformation).filter(Transformation.id == transform_id).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            state=db_transform.pipeline_state.pipeline_state_type.name,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            dataset=db_transform.transformation_template.name)


# CORE Cartridge Notebook::accredo_cancel_before_active
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do. 
![caution](assets/cautionTape.png)

### Configuration

In [30]:
""" 
********* VARIABLES - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<variable_name>: <data_type> #<comment explaining what the value is to future us>

e.g.

class Transform(DbTransform):
    some_ratio: float
    site_name: str

~~These ARE the droids you are looking for~~
"""

class Transform(DbTransform):
    '''
    YOUR properties go here!!
    Variable properties should be assigned to the exact name of
    the transformation as it appears in the Jupyter notebook filename.
    '''
    # Column headers
    status_date: str = 'status_date'# Status date column
    ref_date: str = 'rec_date'# Referral date column
    patient: str = 'msa_patient_id' # Patient ID column
    pharm: str = 'pharm_code' # Pharmacy Name column
    product: str = 'medication' # Medication Name column
    status: str = 'status_code' # Status column
    substatus: str = 'sub_status' # Substatus column
    ic_status: str = 'integrichain_status' # IntegriChain Status column
    ic_substatus: str = 'integrichain_sub_status' # IntegriChain Substatus column
    pjh: str = 'Patient_Journey_Hierarchy' # Patient Journey Hierarchy column
    brand: str = 'brand' # Brand column (medication without strength)
        
    # Possible status values
    pending: str = 'PENDING' # Pending status 'PENDING'
    active: str = 'ACTIVE' # Active status 'ACTIVE'
    cancelled: str = 'CANCELLED' # Cancelled status 'CANCELLED'
    discontinued: str = 'DISCONTINUED' # Discontinued status 'DISCONTINUED'
        
    # Possible substatus values
    pending_new: str = 'NEW' # New substatus when status is 'PENDING'
    active_shipped: str = 'SHIPMENT'# Shipment substatus when status is 'ACTIVE'
    
    # Possible PJH values
    bvpa: str = 'BV/PA' # BV/PA pjh
    intake: str = 'Intake' # Intake pjh
    fulfillment: str = 'Fulfillment' # Fulfillment pjh
    transfer: str = 'Transferred' # Transferred pjh
    payer: str = 'Payer' # Payer pjh
    no_clarity: str = 'No Status Clarity' # Final result of enrichment

transform = Transform()

In [31]:
## Please place your value assignments for development here!!
## This cell will be turned off in production and Engineering will set to pull from the configuration application instead
## For the last example, this could look like...
## transform.some_ratio = 0.6
## transform.site_name = "WALGREENS"

# Vars
patient = 'msa_patient_id'
pharm = 'pharm_code'
product = 'medication'
ref_date = 'rec_date'
status_date = 'status_date'
status = 'status_code'
substatus = 'sub_status'
ic_status = 'integrichain_status'
ic_substatus = 'integrichain_sub_status'
pjh = 'Patient_Journey_Hierarchy'
brand = 'brand'

# Values
pending = 'PENDING'
active = 'ACTIVE'
cancelled = 'CANCELLED'
discontinued = 'DISCONTINUED'
pending_new = 'NEW'
active_shipped = 'SHIPMENT'
bvpa = 'BV/PA'
intake = 'Intake'
fulfillment = 'Fulfillment'
transfer = 'Transferred'
payer = 'Payer'
no_clarity = 'No Status Clarity'

### Description
What does this transformation do? be specific.

![what does your transform do](assets/what.gif)

### Transformation

In [7]:
"""
************ FETCH DATA - TOUCH, BUT CAREFULLY **************
This cell will be turned off in production, as the input_contract will be handled by the pipeline.
"""

if not input_branch:
    input_branch = BRANCH_NAME
input_contract = DatasetContract(branch=input_branch, state=input_state, parent=input_pharma, child=input_brand, dataset=input_name)
run_filter = []
run_filter.append(dict(partition="__metadata_run_id", comparison="==", values=[3]))
# IF YOU HAVE PUBLISHED DATA MULTIPLE TIMES, uncomment the above line and change the int to the run_id to fetch.
# Otherwise, you will have duplicate values in your fetched dataset!
final_dataframe = input_contract.fetch(filters=run_filter)

2019-08-13 21:47:03,647 - core.dataset_contract.DatasetContract - INFO - Fetching dataframe from s3 location s3://ichain-dev/sun-extract-validation/sun/ilumya/ingest/symphony_health_association_ingest_column_mapping.


### Copy ingested data

In [8]:
df = final_dataframe.copy()

### Impute PJH and Preprocess Data

In [9]:
"""
This cell brings in the Patient Journey Hierarchy information from a local file.
This is just a placeholder to ensure data processes work as expected until the
true ingest data can be used.
"""

import os

pd.options.display.max_columns=999

os.chdir('{}'.format(os.path.expanduser('~')))
status_config = pd.read_csv('status_mapping.csv')

status_config = (
    status_config
    .assign(**{
        status: lambda x:(
            x['statusCode'].str.upper()
        ),
        substatus: lambda x:(
            x['subStatus'].str.upper()
        ),
        ic_status: lambda x:(
            x['integrichain_status'].str.upper()
        ),
        ic_substatus: lambda x:(
            x['integrichain_sub_status'].str.upper()
        ),
        pjh: lambda x:(
            x['Patient_Journey_Hierarchy'].str.upper()
        )
    })
    .drop(columns=[
        'statusCode',
        'subStatus'
    ])
)

df = (
    df
    .loc[:,
        [ref_date,
         patient,
         pharm,
         product,
         status_date,
         status,
         substatus]
        ]
    .fillna('Null')
    .assign(**{
        brand: lambda x:(
            x[product].apply(lambda x: x.split()[0].strip())
        ),
        status: lambda x:(
            x[status].str.upper()
        ),
        substatus: lambda x:(
            x[substatus].str.upper()
        )
    })
    .assign(**{
        status_date: lambda x:(
            pd.to_datetime(
                x[status_date].str[:8].astype(str),
                errors='coerce'
        )),
        ref_date: lambda x:(
            pd.to_datetime(
                x[ref_date].str[:8].astype(str),
                errors='coerce'
        )),
        'min_status_date': lambda x:(
            x.groupby([patient, pharm, brand])[status_date].transform(min)
        )
    })
    .fillna(value={ref_date: 'min_status_date'})
    .drop(columns=['min_status_date'])
    .merge(status_config, how='left', on=[status, substatus])
    .drop_duplicates()
    .sort_values(
        [patient, pharm, brand, status_date]
    )
)

df.loc[df[brand] == 'Null', [brand, product]] = np.nan

### Transformation

In [10]:
# Sort and reset index to join on later
df = (
    df
    .sort_values([patient, pharm, brand, status_date, ref_date])
    .reset_index(drop=False)
)

In [11]:
# Create max status date column for each patient journey
df = (
    df
    .assign(max_status_date=lambda x:(
        x.groupby([patient, pharm, brand])[status_date].transform(max)
    ))
)

In [12]:
# Find difference in days between status date and max status date
df = (
    df
    .assign(day_difference=lambda x:(
        x['max_status_date'] - x[status_date]
    ))
)

In [21]:
# Drop records from df to find last status and substatus
last_status_df = (
    df
    .sort_values([patient, pharm, brand, status_date, ref_date])
    # Also create previous status and substatus columns
    .assign(prev_status=lambda x:(
        x[ic_status].shift(-1)
    ))
    .assign(prev_substatus=lambda x:(
        x[ic_substatus].shift(-1)
    ))
    .drop_duplicates(subset=[patient, pharm, brand], keep='last')
    [[patient, pharm, brand, status_date, ref_date, ic_status, ic_substatus, 'prev_status', 'prev_substatus']]
    .rename(columns={ic_status: 'last_status', ic_substatus: 'last_substatus'})
    .drop(columns=[patient, pharm, brand, status_date, ref_date])
)

In [26]:
df.columns

Index(['index', 'rec_date', 'msa_patient_id', 'pharm_code', 'medication',
       'status_date', 'status_code', 'sub_status', 'brand',
       'integrichain_sub_status', 'integrichain_status',
       'Patient_Journey_Hierarchy', 'max_status_date', 'day_difference',
       'last_status', 'last_substatus', 'prev_status', 'prev_substatus'],
      dtype='object')

In [23]:
# Merge to df to obtain those two columns
df = (
    df
    .join(last_status_df, how='left')
)

In [29]:
# Create column denoting prev status and substatus that are in one of the three cases
df = (
    df
    .assign(prev_check=lambda x:(
        True if (
            (x.prev_status == pending and x.prev_substatus == pending_new) |
            (x.prev_status.isin([cancelled, discontinued]))
        ) else False
    ))
)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
# Apply enrichment using loc with the case structure
df.loc[(
    (df['prev_check' == True]) &
    (
        (df['last_status' == active]) &
        (df['last_substatus' == active_shipped])
    ) &
    (df['day_difference' > 2])
), pjh] = 'No Status Clarity'

In [17]:
### Use the variables above to execute your transformation. the final output needs to be a variable named final_dataframe

### Publish

In [18]:
## that's it - just provide the final dataframe to the var final_dataframe and we take it from there
transform.publish_contract.publish(final_dataframe, run_id, session)
session.close()

2019-08-13 21:31:13,983 - core.dataset_contract.DatasetContract - INFO - Publishing dataframe to s3 location s3://ichain-dev/ds301_accredo_enrichment/sun/ilumya/enrich/accredo_cancel_before_active with run ID 403817.
2019-08-13 21:31:13,995 - core.dataset_contract.DatasetContract - DEBUG - Publishing dataframe to Redshift Spectrum database ichain_core to schema.table                 data_core.sun_ilumya_accredo_cancel_before_active...
2019-08-13 21:31:14,325 - s3parq.publish_redshift - INFO - Running query to create schema: CREATE EXTERNAL SCHEMA IF NOT EXISTS data_core                 FROM DATA CATALOG                 database 'ichain_core'                 iam_role 'arn:aws:iam::265991248033:role/mySpectrumRole';


/usr/local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


ProgrammingError: (psycopg2.ProgrammingError) permission denied for database ichain_core

[SQL: CREATE EXTERNAL SCHEMA IF NOT EXISTS data_core                 FROM DATA CATALOG                 database 'ichain_core'                 iam_role 'arn:aws:iam::265991248033:role/mySpectrumRole';]
(Background on this error at: http://sqlalche.me/e/f405)